# Linear Regression for Solar Installation Prediction

This notebook demonstrates linear regression for both classification and regression using the solar installation data.

## Setup

### Load packages and data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, mean_squared_error, r2_score

In [ ]:
# Base URL for raw GitHub content
base_url = "https://raw.githubusercontent.com/chuckgrigsby0/agec-784/main/data/"

# Load solar directly from GitHub URL
solar_data = pd.read_csv(base_url + 'solar-data.csv')

print("Data loaded successfully!")
print(f"Number of rows and columns: {solar_data.shape}")

### Data exploration

In [ ]:
# Print the column names
# Note that .columns is an attribute of solar_data
print(solar_data.columns)

In [ ]:
# Print the first 5 rows of the dataset
print(solar_data.head())

In [ ]:
# Compute summary statistics, rounded to 4 decimal places 
# Note: only numeric columns are included
np.round(solar_data.describe(), decimals=4)

In [ ]:
# To get counts of number of households that installed solar or not, we can use the value_counts() method
solar_data['Install?'].value_counts()

In [ ]:
# Create binary numeric variable for modeling
# Many regression models require numeric (0/1) rather than categorical (Yes/No) target variables
i = solar_data.columns.get_loc('Install?') + 1
solar_data.insert(i, 'Install', np.where(solar_data['Install?'] == 'Yes', 1, 0))

In [ ]:
solar_data.head()

## Linear Probability Model for Classification

We will estimate a linear probability model to predict whether a household installs solar panels (Yes/No) based on Income and Peak Sun Hours (PSH). Although the linear probability model produces predicted probabilities outside the [0, 1] range, it is a useful starting point for understanding the relationship between predictors and the binary outcome.

### Prepare the data

Split the data into train and test sets. 

In [ ]:
# Split data into training (70%) and testing (30%) sets
# We evaluate model performance using test data to assess how well the model generalizes to unseen observations
# random_state ensures reproducibility (the same split each time the code runs)
train_data, test_data = train_test_split(
    solar_data,
    train_size=0.7,
    test_size=0.3,
    random_state=731
)

### Estimate the Linear Probability Model (LPM)

In [ ]:
lpm_train = smf.ols(formula='Install ~ Income + PSH', data=train_data).fit()

In [ ]:
# Display the summary results
lpm_train.summary()

### Make predictions

We will convert predicted probabilities to class labels (0 or 1) using a threshold of 0.5. The LPM predicts the probability of solar installation, and we classify as '1' (install) if the predicted probability is 0.5 or higher, otherwise '0' (no install).

In [ ]:
# Generate predictions once and store for efficiency
test_pred_probs = lpm_train.predict(test_data)

# Check for out-of-bounds predictions
print(f"Predictions < 0: {(test_pred_probs < 0).sum()}")
print(f"Predictions > 1: {(test_pred_probs > 1).sum()}")

In [ ]:
# Predict on test set
preds_lpm_df = pd.DataFrame({
    'pred_prob': test_pred_probs, 
    'pred': np.where(test_pred_probs >= 0.5, 1, 0),
    'actual': test_data['Install']
})

In [ ]:
preds_lpm_df.head()

### Assessing Model Accuracy

Accuracy = 1 - Misclassification Rate

In [ ]:
# All yield the same result

print(f"Accuracy: {accuracy_score(preds_lpm_df['actual'], preds_lpm_df['pred']):.4f}") # Using sklearn.metrics

print(f"Accuracy: {1 - np.mean(preds_lpm_df['actual'] != preds_lpm_df['pred']):.4f}") # Manual calculation of accuracy

print(f"Accuracy: {np.mean(preds_lpm_df['actual'] == preds_lpm_df['pred']):.4f}") # Manual calculation of accuracy

#### Confusion Matrix

In [ ]:
cm = confusion_matrix(preds_lpm_df['actual'], preds_lpm_df['pred'], labels=[0, 1])
print(f"\nConfusion Matrix:\n{cm}")

In [ ]:
# Visualization of Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot();

## Linear Regression for Predicting Net Profit

In [ ]:
reg_train = smf.ols(formula='Profit ~ Income + PSH', data=train_data).fit()

In [ ]:
# Display the summary results
reg_train.summary()

### Make predictions

In [150]:
test_preds = reg_train.predict(test_data)

# Predict on test set
preds_reg_df = pd.DataFrame({
    'pred': test_preds, 
    'actual': test_data['Profit']
})

In [ ]:
preds_reg_df.head()

### Evaluate model performance

$\text{MSE}$ (Mean Squared Error) measures the average squared difference between actual and predicted values. Lower values indicate better model performance.

$\text{R}^{2}$ ranges from 0 to 1, where 1 indicates perfect predictions and 0 means the model performs no better than predicting the mean.

$\text{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2$

$R^2 = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y})^2}$

Where: $y_i$ = actual net profit, $\hat{y}_i$ = predicted net profit, $\bar{y}$ = average net profit, n = number of observations

In [151]:
# Evaluate
mse = mean_squared_error(preds_reg_df['actual'], preds_reg_df['pred'])
r2 = r2_score(preds_reg_df['actual'], preds_reg_df['pred'])

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R²): {r2:.4f}")

Mean Squared Error (MSE): 7.5132
R-squared (R²): 0.8182


In [152]:
# All yield the same result
numerator = np.sum((preds_reg_df['actual'] - preds_reg_df['pred']) ** 2)
denominator = np.sum((preds_reg_df['actual'] - np.mean(preds_reg_df['actual'])) ** 2)

print(f"MSE: {np.mean((preds_reg_df['actual'] - preds_reg_df['pred']) ** 2): .4f}") # Manual calculation of MSE
print(f"R-squared: {1 - numerator / denominator: .4f}") # Manual calculation of R^2

MSE:  7.5132
R-squared:  0.8182


### Advanced: Visualizing Partial Regression Effects

Partial regression plots show the relationship between the outcome and an explanatory variable, purging the influence of the other explanatory variables from both the outcome and the explanatory variable of interest. This helps visualize the unique contribution of each predictor.

Reference: [NIST - Partial Regression Plot](https://www.itl.nist.gov/div898/software/dataplot/refman1/auxillar/partregr.htm)

In [ ]:
# Partial regression plots for both predictors
fig = sm.graphics.plot_partregress_grid(reg_train, 
                                        exog_idx=['Income', 'PSH'],
                                        grid=(1, 2), 
                                        fig=plt.figure(figsize=(12, 4)))
plt.tight_layout()
plt.show()

In [ ]:
# Partial regression plot for Income only
fig, ax = plt.subplots(figsize=(8, 6))
sm.graphics.plot_partregress('Profit', exog_i='Income', exog_others=['PSH'], 
                             data=train_data, obs_labels=False, ax=ax)
ax.set_title('Partial Regression: Profit vs. Income (controlling for PSH)')
plt.show()